In [1]:
from google.colab import files
files.upload()   # Upload kaggle.json here



Output hidden; open in https://colab.research.google.com to view.

In [1]:
import pandas as pd

df = pd.read_csv("BitcoinHeistData.csv.zip")   # change path if inside folder
print("✅ Dataset loaded successfully!")
print("Shape of dataset:", df.shape)  # shows rows & columns
df.head()  # displays first 5 rows


✅ Dataset loaded successfully!
Shape of dataset: (2916697, 10)


,address,year,day,length,weight,count,looped,neighbors,income,label
0,111K8kZAEnJg245r2cM6y9zgJGHZtJPy6,2017,11,18,0.008333,1,0,2,100050000.0,princetonCerber
1,1123pJv8jzeFQaCV4w644pzQJzVWay2zcA,2016,132,44,0.000244,1,0,1,100000000.0,princetonLocky
2,112536im7hy6wtKbpH1qYDWtTyMRAcA2p7,2016,246,0,1.000000,1,0,2,200000000.0,princetonCerber
3,1126eDRw2wqSkWosjTCre8cjjQW8sSeWH7,2016,322,72,0.003906,1,0,2,71200000.0,princetonCerber
4,1129TSjKtx65E35GiUo4AYVeyo48twbrGX,2016,238,144,0.072848,456,0,1,200000000.0,princetonLocky


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# --- Load dataset ---
df = pd.read_csv("BitcoinHeistData.csv.zip")

# --- Features & Target ---
X = df.drop("label", axis=1)
y = df["label"]

# Binary classification: Ransomware (1) vs Benign (0)
y = y.apply(lambda x: 0 if x == "benign" else 1)

# Encode categorical columns using LabelEncoder (more memory-efficient than one-hot)
for col in X.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# --- Train-test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- HistGradientBoosting Model (faster & memory-friendly) ---
model = HistGradientBoostingClassifier(
    max_iter=100, learning_rate=0.1, max_depth=5, random_state=42
)
model.fit(X_train, y_train)

# --- Predictions ---
y_pred = model.predict(X_test)

# --- Accuracy ---
accuracy = accuracy_score(y_test, y_pred)
print("✅ HistGradientBoosting Model Accuracy:", accuracy)


✅ HistGradientBoosting Model Accuracy: 1.0


In [8]:
from sklearn.metrics import classification_report

# Print classification metrics
print("✅ HistGradientBoosting Model Accuracy:", accuracy)
print("\n📊 Classification Report:")

# Specify labels explicitly to match target_names
print(classification_report(
    y_test,
    y_pred,
    labels=[0, 1],  # 0 = Benign, 1 = Ransomware
    target_names=["Benign", "Ransomware"]
))


✅ HistGradientBoosting Model Accuracy: 1.0

📊 Classification Report:
              precision    recall  f1-score   support

      Benign       0.00      0.00      0.00         0
  Ransomware       1.00      1.00      1.00    583340

    accuracy                           1.00    583340
   macro avg       0.50      0.50      0.50    583340
weighted avg       1.00      1.00      1.00    583340



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/me

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# --- Load dataset ---
df = pd.read_csv("BitcoinHeistData.csv.zip")

# --- Features & Target ---
X = df.drop("label", axis=1)
y = df["label"]

# Binary classification: Ransomware (1) vs Benign (0)
# Correcting the mapping based on the actual unique values ('white' for benign)
y = y.apply(lambda x: 0 if x == "white" else 1)

# Display value counts of the target variable to check class distribution
print("Target variable value counts after transformation:\n", y.value_counts())

# Encode categorical features using LabelEncoder
for col in X.select_dtypes(include='object').columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# --- Train-test split (stratified to maintain class distribution) ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- Handle class imbalance using SMOTE ---
smote = SMOTE(random_state=42)
X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

# --- HistGradientBoosting Model ---
model = HistGradientBoostingClassifier(
    max_iter=100, learning_rate=0.1, max_depth=5, random_state=42
)
model.fit(X_train_bal, y_train_bal)

# --- Predictions ---
y_pred = model.predict(X_test)

# --- Accuracy & Classification Metrics ---
accuracy = accuracy_score(y_test, y_pred)
print("✅ HistGradientBoosting Model Accuracy:", accuracy)
print("\n📊 Classification Report:")
print(classification_report(
    y_test,
    y_pred,
    labels=[0, 1],
    target_names=["Benign", "Ransomware"]
))

# --- Optional: Test set class distribution ---
print("\nTest set class distribution:\n", y_test.value_counts())

Target variable value counts after transformation:
 label
0    2875284
1      41413
Name: count, dtype: int64
✅ HistGradientBoosting Model Accuracy: 0.9091610381595638

📊 Classification Report:
              precision    recall  f1-score   support

      Benign       0.99      0.91      0.95    575057
  Ransomware       0.09      0.60      0.16      8283

    accuracy                           0.91    583340
   macro avg       0.54      0.76      0.56    583340
weighted avg       0.98      0.91      0.94    583340


Test set class distribution:
 label
0    575057
1      8283
Name: count, dtype: int64


In [13]:
# Display unique values of the original 'label' column
print("Unique values in the original 'label' column:")
print(df['label'].unique())

Unique values in the original 'label' column:
['princetonCerber' 'princetonLocky' 'montrealCryptoLocker'
 'montrealCryptXXX' 'paduaCryptoWall' 'montrealWannaCry'
 'montrealDMALockerv3' 'montrealCryptoTorLocker2015' 'montrealSamSam'
 'montrealFlyper' 'montrealNoobCrypt' 'montrealDMALocker' 'montrealGlobe'
 'montrealEDA2' 'paduaKeRanger' 'montrealVenusLocker' 'montrealXTPLocker'
 'paduaJigsaw' 'montrealGlobev3' 'montrealJigSaw' 'montrealXLockerv5.0'
 'montrealXLocker' 'montrealRazy' 'montrealCryptConsole'
 'montrealGlobeImposter' 'montrealSam' 'montrealComradeCircle'
 'montrealAPT' 'white']
